In [1]:
from typing_extensions import TypedDict
from dotenv import load_dotenv
import random
from typing import Literal

In [2]:
load_dotenv()

True

In [3]:
class State(TypedDict) :
    graph_info : str 

In [15]:
def start_play(state : State) :

    print("Start_Play Node has been called.")
    return {"graph_info" : state["graph_info"] + " I like to play" }



def cricket(state : State) :

    print("cricket Node has been called.")
    return {"graph_info" : state["graph_info"] + " cricket" }


def badminton(state : State) :

    print("badminton Node has been called.")
    return {"graph_info" : state["graph_info"] + " badminton" }

In [16]:
def random_play(state : State) -> Literal["cricket" , "badminton"] :

    if random.random() > 0.5 :
        return "cricket"
    else :
        return "badminton"

# Building graph

In [17]:
from langgraph.graph import StateGraph , START , END
from IPython.display import display , Image

In [18]:
## Build StateGraph

graph = StateGraph(State)


# Adding Nodes
graph.add_node("start_play" , start_play)
graph.add_node("cricket" , cricket )
graph.add_node("badminton", badminton)

In [19]:
# Connections 

graph.add_edge(START, "start_play")
graph.add_conditional_edges("start_play", random_play)

graph.add_edge("cricket", END)
graph.add_edge("badminton",END)

graph_builder = graph.compile()

In [39]:
graph_builder.invoke({"graph_info" : "Hi I'm Akhilesh"})

Start_Play Node has been called.
badminton Node has been called.


{'graph_info': "Hi I'm Akhilesh I like to play badminton"}

# Stateclass with Pydantic

In [40]:
from langgraph.graph import START , END , StateGraph
from pydantic import BaseModel

In [47]:
class State(BaseModel) :
    name : str

In [58]:
def playground(state : State) :

    return {"name" : state.name }

In [59]:
graph = StateGraph(State)

In [60]:
# add nodes

graph.add_node("playground" , playground)

# add edges

graph.add_edge(START , "playground")
graph.add_edge( "playground" , END )

graph_builder = graph.compile()

In [61]:
graph_builder.invoke( {"name" : "Akhilesh"})

{'name': 'Akhilesh'}

# Tools

In [4]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from typing_extensions import Annotated

from langgraph.graph import StateGraph , START , END

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from pydantic import BaseModel

In [6]:
load_dotenv()

True

In [7]:
llm = ChatGroq(model= "openai/gpt-oss-20b")

In [8]:
def add(a : int , b : int) -> int :

    """_summary_

    Args:
        a (int): first int
        b (int): second int

    Returns:
        int: _description_
    """


    return a+b

In [9]:
llm_with_tools = llm.bind_tools([add])

In [10]:
llm_with_tools.invoke("What is 2+2")

AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to use the function add. So call functions.add with a=2,b=2.', 'tool_calls': [{'id': 'fc_ceda308b-8b02-4d86-99b2-72fc70313f44', 'function': {'arguments': '{"a":2,"b":2}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 131, 'total_tokens': 178, 'completion_time': 0.046346197, 'prompt_time': 0.097291574, 'queue_time': 0.008131504, 'total_time': 0.143637771, 'completion_tokens_details': {'reasoning_tokens': 21}}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_3417a662f8', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--c9b1e4bd-10e5-45dc-9d2d-6b7bfbf361af-0', tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 2}, 'id': 'fc_ceda308b-8b02-4d86-99b2-72fc70313f44', 'type': 'tool_call'}], usage_metadata={'input_tokens': 131, 'output_tokens': 47, 'total_tokens': 178, 'output_

## State

In [20]:
from typing_extensions import TypedDict , Annotated
from langgraph.graph.message import add_messages

from langchain.messages import HumanMessage , AIMessage , AnyMessage

In [12]:
class State(TypedDict) :
    messages : Annotated[ list[AnyMessage] , add_messages ]

In [13]:
graph = StateGraph(State)

In [14]:
def llm_tool(state : State) :

    return {"messages" : [ llm_with_tools.invoke(state["messages"]) ] }

In [15]:
graph.add_node("llm_tool" , llm_tool)


graph.add_edge(START , "llm_tool")
graph.add_edge("llm_tool" , END)

graph_builder = graph.compile()

In [ ]:
messages = graph_builder.invoke({"messages" : ["What is 6+3922929"] })

{'messages': [HumanMessage(content='What is 6+3922929', additional_kwargs={}, response_metadata={}, id='7edfb158-e5cd-46a1-bbd0-2ee17b1329fd'),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'User asks: "What is 6+3922929". That\'s a simple addition: 3,922,929 + 6 = 3,922,935. We can compute. We might use the add function. Let\'s use the function.', 'tool_calls': [{'id': 'fc_21341a71-262f-4928-b726-415c30758bd6', 'function': {'arguments': '{"a":6,"b":3922929}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 133, 'total_tokens': 213, 'completion_time': 0.079922365, 'prompt_time': 0.006560398, 'queue_time': 0.008093967, 'total_time': 0.086482763, 'completion_tokens_details': {'reasoning_tokens': 52}}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_d08f3e4cc0', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--016cf17f-a3a9

# Tool Node for output display

In [22]:
from langgraph.prebuilt import ToolNode , tools_condition

from typing_extensions import TypedDict , Annotated
from langgraph.graph.message import add_messages

from langchain.messages import HumanMessage , AIMessage , AnyMessage

In [23]:
class State(TypedDict) :
    messages : Annotated[ list[AnyMessage] , add_messages ]

In [28]:
def add(a : int , b : int) -> int :

    """_summary_

    Args:
        a (int): first int
        b (int): second int

    Returns:
        int: _description_
    """


    return a+b

In [29]:
llm_with_tools = llm.bind_tools([add])

In [26]:
def llm_tool(state : State) :

    return {"messages" : [ llm_with_tools.invoke(state["messages"]) ] }

In [68]:
graph = StateGraph(State)

In [75]:
tool = [add]

In [ ]:
graph.add_node("llm_tool" , llm_tool)
graph.add_node("tools" , ToolNode(tool))



graph.add_edge(START , "llm_tool")

graph.add_conditional_edges("llm_tool", tools_condition)

graph.add_edge("tools",END)





graph_builder = graph.compile()

In [74]:
graph_builder.invoke( { "messages" :   "What is 2025+444444" } )

{'messages': [HumanMessage(content='What is 2025+444444', additional_kwargs={}, response_metadata={}, id='0a679ae6-1c67-4a97-8b3d-a2b0853143d4'),
  AIMessage(content='', additional_kwargs={'reasoning_content': "We need to add 2025 + 444444. That is 446,469? Let's compute: 444,444 + 2,025 = 446,469. Yes. But we have a tool add that expects a and b. We can use it.", 'tool_calls': [{'id': 'fc_15f9d9fe-a79d-4636-9490-3d143c9b4897', 'function': {'arguments': '{"a":2025,"b":444444}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 133, 'total_tokens': 218, 'completion_time': 0.083965086, 'prompt_time': 0.006474908, 'queue_time': 0.00808851, 'total_time': 0.090439994, 'completion_tokens_details': {'reasoning_tokens': 57}}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_017d3da75a', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--8069ece2-b219-